In [4]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [6]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [8]:
tool.name

'wikipedia'

In [12]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")

In [14]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001"
)

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vector_db = FAISS.from_documents(documents, embeddings)
retriever = vector_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000021C21A1E910>)

In [15]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool.")

In [16]:
retriever_tool.name

'langsmith_search'

In [17]:
# Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [18]:
tools = [tool, arxiv, retriever_tool]

In [19]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Akshobh\\PROGRAMMING\\Python\\Tooling\\LangChain\\Tutorial\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool.', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000021CCAC776A0>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenera

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
  model="gemini-2.5-flash-lite",
)
llm

ChatGoogleGenerativeAI(model='gemini-2.5-flash-lite', client=genai.GenerativeModel(
    model_name='models/gemini-2.5-flash-lite',
    generation_config={},
    safety_settings={},
    tools=None,
))

In [39]:
from langchain_core.prompts import PromptTemplate

# Create a ReAct prompt template compatible with Gemini
prompt_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad", "tools", "tool_names"],
    template=prompt_template
)

In [40]:
prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [41]:
### Agents
from langchain.agents import create_react_agent, AgentExecutor

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': 'wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv: A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith_search: Search for information about LangSmith. For any questions about LangSmith, you must use this tool.', 'tool_names': 'wikipedia, arxiv, langsmith_search'}, template='Answer the following questions as best you can. You have a

In [42]:
agent_executor.invoke({"input":"Tell me about LangSmith"})



> Entering new AgentExecutor chain...
I need to find information about LangSmith. The `langsmith_search` tool is specifically designed for this purpose.
Action: langsmith_search
Action Input: LangSmithLangSmith is framework agnostic, so you can use it with or without LangChain’s open-source libraries
langchain and langgraph.
Prototype locally, then move to production with integrated monitoring and evaluation to build more reliable AI systems.
LangGraph Platform is now LangSmith Deployment. For more information, check out the Changelog.
​Get started
Create an accountSign up at smith.langchain.com (no credit card required).
You can log in with Google, GitHub, or email.Create an API keyGo to your Settings page → API Keys → Create API Key.
Copy the key and save it securely.
Once your account and API key are ready, choose a quickstart to begin building with LangSmith:

LangSmith docs - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KSupportGitHubTry Lan

{'input': 'Tell me about LangSmith',
 'output': "LangSmith is a platform that provides tools for developing, debugging, and deploying LLM applications. It helps users trace requests, evaluate outputs, test prompts, and manage deployments. LangSmith is framework-agnostic, meaning it can be used with or without LangChain's open-source libraries like `langchain` and `langgraph`. It offers features such as observability, evaluation, and deployment, and can be set up in managed cloud, self-hosted, or hybrid environments. LangSmith also includes Agent Builder for visual agent design and Studio for end-to-end application development. It emphasizes data security and privacy, complying with HIPAA, SOC 2 Type 2, and GDPR. To get started, users can create an account and an API key on smith.langchain.com."}

In [43]:
agent_executor.invoke({"input":"Tell me about Machine Learning"})



> Entering new AgentExecutor chain...
I need to find information about Machine Learning. Wikipedia is a good source for general information on this topic.
Action: wikipedia
Action Input: Machine LearningPage: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genI have found a summary of Machine Learning from Wikipedia. I can now answer the question.
Final Answer: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize.

> Finished chain.


{'input': 'Tell me about Machine Learning',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize.'}

In [44]:
agent_executor.invoke({"input":"What's the paper 1605.08386 all about?"})



> Entering new AgentExecutor chain...
I need to find out what the paper with ID 1605.08386 is about. The `arxiv` tool is suitable for this purpose as it can search for scientific articles. I will use the paper's ID as the search query.
Action: arxiv
Action Input: 1605.08386

d:\Akshobh\PROGRAMMING\Python\Tooling\LangChain\Tutorial\.venv\Lib\site-packages\langchain_community\utilities\arxiv.py:149: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  ).results()


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of alloThe user is asking about the content of a specific paper identified by its arXiv ID. The `arxiv` tool has already been used to retrieve the title, authors, and a summary of the paper. This information should be sufficient to answer the question.
Final Answer: The paper 1605.08386, titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch, is about studying graphs on lattice points whose edges come from a finite set of allowed steps.

> Finished chain.


{'input': "What's the paper 1605.08386 all about?",
 'output': 'The paper 1605.08386, titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch, is about studying graphs on lattice points whose edges come from a finite set of allowed steps.'}